In [1]:
import pandas as pd
import numpy as np

# Summarizing, Aggregating & Grouping
Knowing how to use pandas aggregation & grouping functions lets us reduce the dimensionality of our data and visualize it in different ways (most often over the rows - aka `axis=0`).  

For completeness, `axis=1` refers to the columns.

In [2]:
wine = pd.read_csv('./data/wine_reviews/winemag-data_first150k.csv', index_col=0)

## Initial quick analysis using pandas
Pandas has multiple built-in functions that make it easy to quickly see what's in your dataframe. 
You can build them with the selecting tools you used before.

Here, we will select a column, and then see how pandas lets us quickly analyse it.

To quickly see which columns our wine dataset has, we can use the .columns attribute.

In [ ]:
wine.columns

Let's select price.

In [ ]:
wine.loc[:, 'price']

We can quickly see some metrics of the price, using some built-in aggregating functions in pandas.

In [ ]:
print('Average wine price: ', wine.loc[:, 'price'].mean())
print('Min wine price: ', wine.loc[:, 'price'].min())
print('Median wine price: ', wine.loc[:, 'price'].median())
print('Max wine price: ', wine.loc[:, 'price'].max())

And some more advanced metrics...

We can see how many times each country appears in the datast (called a value count)

In [ ]:
wine.loc[:, 'country'].value_counts().sort_index()

What if we want to see this list in the opposite order?

In [ ]:
wine.loc[:, 'country'].value_counts().sort_values(ascending=True)

This list is too long. What if we only want to see the top 10 countries?
We can string together the other selectors we learned before!

Can you think of another way to get the top 10 rows?

In [ ]:
wine.loc[:, 'country'].value_counts().head(10)

In [ ]:
wine.loc[:, 'country'].value_counts().iloc[:10]

What's the difference between `.value_counts()` and `.count()`?

In [ ]:
wine.loc[:, 'country'].count()

What if we just want to know how many countries are on the list?

In [ ]:
# Number of non-null unique values
wine.loc[:, 'country'].nunique()

And if we want a list of them? 
(This isn't a pandas thing, but is still super useful:)

In [ ]:
# Gives all unique values
set(wine['country'])

You can look [here](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/) for a list of all the built-in pandas stats.

Remember, for quick calculation of summary statistics we can use `df.describe()`. This quickly calculates some of these stats for the numeric columns in the df.

In [ ]:
wine.describe()

**Question**: Why are only 2 of the columns included?

### Conditional Selections 
We can use conditional selections to narrow our analysis even further.

DON'T FORGET - to make things easier, we can save selections we plan to use often as their own variables.

In [14]:
us = wine.loc[wine.loc[:, 'country']=='US']
france = wine.loc[wine.loc[:, 'country']=='France']

Let's compare the difference in average price between US wines, French wines, and all the wines in our complete dataset.

In [ ]:
# calculating average wine prices 
avg_us_price = round(us.loc[:, 'price'].mean(),2)
avg_french_price = round(france.loc[:, 'price'].mean(),2)
avg_overall_price = round(wine.loc[:, 'price'].mean(),2)

# printing prices

print('Mean American wine price: $', avg_us_price)
print('Mean French wine price: $', avg_french_price)
print('Mean overall wine price is: $', avg_overall_price)

We can then use these to calculate more targeted metrics.

In [ ]:
print('French wine is ${} more expensive on average'.format(round(avg_french_price - avg_overall_price), 2))

#### More advanced conditionals: Using masks
What if we want to filter on more than one criteria? 

When you want to filter on >1 criteria, it can be easier to use a mask.

How many wines from North America do we have on our list?

In [17]:
na_mask = (wine.country == 'US') | (wine.country == 'Mexico') | (wine.country == 'Canada') 
na_indexes = wine.index[na_mask]
na = wine.loc[na_indexes]
# or
na_1 = na.loc[na_mask]

In [ ]:
na.head()

In [ ]:
na_1.head()

How many wines do we have total in the in North America?

In [ ]:
# taking one of the columns that had no null values to get "real" count
na.count()['description']

**Question**: How many of the wines belong to each country?



In [ ]:
na.loc[:, "country"].value_counts()

**Question**: From which US state do most of our wines come?

In [ ]:
us.loc[:, "province"].value_counts().sort_values(ascending=False).head(1)

## Groupby
One of the most flexible ways to aggregate in pandas is with .groupby() .
We will look at how this works for categorical datasets like this one, and also for datetime datasets, as dealing with datetimes in pandas can be tricky.

In [23]:
import matplotlib.pyplot as plt
%matplotlib inline

### How Groupby Works:
You can group your data in many different ways, and also aggregate it by any of the aggregators we saw before: like mean, mode, sum, etc.

**The column that you put in the parentheses of the groupby function call will become your new index in the dataframe that is returned by groupby!!**

For example, in `wine.groupby('country')`, `country` will bee the new index. Let's see how this looks in practice:

In [ ]:
wine.groupby('country').mean().sort_values(by=['points','price'], ascending=False)

So, British wine is the highest average ranked?? This is surprising. Maybe we need to look at our data in a different way.

The beauty of `df.groupby()` is that it lets you aggregate different columns in different ways.

Say we want to know the average price of wine in each country, but the _highest_ score:

In [ ]:
wine.groupby('country').agg({'points':'max', 'price':'mean'})\
    .sort_values(by=['points','price'], ascending=[False, True]).head()

**Tip:** Sometimes, to make your code cleaner it's best to move the aggregations out and store them in their own variable that you can update saparately.

In [ ]:
aggs = {
    'price': 'mean',
    'points': 'max'
}

wine.groupby('country').agg(aggs).sort_values(by=['points','price'], ascending =[False, True]).head()

You can also use a list in your aggs to aggergate one column in different ways.

This will give a **multi-index**. Multi-indexes can be difficult to sort on. But, there are a few different ways we can deal with this.

In [ ]:
aggs = {
    'price': ['min', 'mean', 'max', 'std']
}

price_table = wine.groupby('country').agg(aggs)
price_table

One way is by dropping the top level ('price'):

In [ ]:
price_table.columns = price_table.columns.droplevel(level=0)

# now, we can sort by any of the columns. Here, by average price.
price_table.sort_values(by='mean', ascending=False).head()

Another way is by using `np.ravel()` . This preserves the "price" indicator somewhere in each of the column names.

In [ ]:
# resetting the price table, so that it again has a multi-index
price_table = wine.groupby('country').agg(aggs)

price_table.head()

In [ ]:
# seeing how the column names are multi-index
price_table.columns

In [ ]:
# Using ravel, and a string join, we can create better names for the columns:

price_table.columns = ["_".join(x) for x in price_table.columns.ravel()]
price_table.sort_values(by='price_mean', ascending=False).head()

**Question**: Create a quick plot of the number of wines made in each country, from highest to lowest.

In [ ]:
wine.loc[:, 'country'].value_counts().plot(kind='bar', figsize=(10,7))
plt.show()

## Selecting the max and min values with Index Max and Min
In addition to `.max()` and `.min()`, which returns the maximum or minimum values, we can use `.idxmax()` and `.idxmin()` to return the *index* pertaining to the maximum and minimum values. 

For example, let's use `.idxmax()` to find the country with the highest standard deviation in its prices.

In [ ]:
price_table.loc[:, 'price_std'].idxmax()

In [ ]:
# looking at the actual standard deviation 
price_table.loc[:, 'price_std'].max()

# Dealing with datetime in Pandas
Pandas built off the datetime package in Python to offer a datetime index, and plenty of ways to work with this.
However, it is still far from intuitive. 
That doesn't mean it's not useful, and for anyone doing a timeseries project, you'll need to deal with dates and times in pandas often.

Let's load a sample dataset of datetime energy data and get started!

In [38]:
energy = pd.read_csv('./data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

Note that this data has a DateTimeIndex. 
setting `parse_dates=True` when we read the CSV lets pandas infer this datetimeindex.

In [ ]:
energy.index

We can select data points within a specific time range, using the DateTimeIndex and .loc.
Here, we select the first one day of data.

In [ ]:
energy.loc['1998-12-31 01:00:00':'1999-01-01 00:00:00']

In [ ]:
# One record for each hour of this day.
energy.loc['1998-12-31 01:00:00':'1999-01-01 00:00:00', :].shape[0]

## Selecting with boolean indexing on pandas datetimeindex
We can use .dot notation with parts of the date to select on specific parts of the datetime, like days or months.

This is because pandas datetime indexes are datetime objects under the hood.

Let's look more closely at datetime objects in python.

Python datetime functionality example:

In [42]:
from datetime import datetime

We will start with a string of a date, and save this string in a variable called `s`.

In [43]:
s = "8 March, 2017"

Using a datetime function called `datetime.strptime()`, we can "extract" the date elements from this string.

We do this by giving formatting indicators as to where in the string the various datetime elements are located. The format indicator `%d` denotes an integer day, `%B` denotes a long-form month written in English, and `%Y` denotes a 4-digit year.

You can find more datetime format indicators in python [here](https://mkaz.blog/code/python-dates/) (under "Datetime Format Table").


Using the `.strptime()` function returns a **datetime object**, with the parts of the datetime contained in order from lowest time-granularity to highest (year --> seconds)

In [ ]:
d = datetime.strptime(s, '%d %B, %Y')
d

As mentioned before, we can access particular parts of this datetime object using dot notation (as they are stored as attributes of the object)

In [ ]:
d.month

We can then use a second function from datetime, called `strftime()`, to convert this datetime object back to a string. Here, we have to specify the new string format we would like the new date string to have.

In [ ]:
new_s = d.strftime('%Y-%m-%d')
print(new_s)

Now that we know about the various parts of a datetime object, we can use them to filter conditionally on the datetime object.

In [47]:
# making a new DF that only includes the month of septmeber from each year.
septembers = energy.loc[energy.index.month == 9]

We can also call just a date, and get all the hours/time periods in that day:

In [ ]:
energy.loc['2000-01-03']

Same with for a year and month:

In [ ]:
energy.loc['2000-01'].shape

In [ ]:
# We see that it includes one record for each our of each day of the month of January, which has 31 days
31*24

In [51]:
# or better, with an assert statement:
month_jan = energy.loc['2000-01']

# assert passes silently, meaning it is True
assert month_jan.shape[0] == 31*24

## Resampling
We can also combine the data in different ways, and over different time periods.
This means that just because our data is in hourly time periods, we dont have to keep it that way. 

In [ ]:
# We can get the average load over a day:
daily_avg_energy = energy.resample('D').mean()
daily_avg_energy.head()

In [ ]:
# We can also get the total MWh used in a day:
daily_energy = energy.resample('D').sum()
# have to change unit from MW to MWh since summing changes measure from power to energy 
# more info on that here: https://energyeducation.ca/encyclopedia/Energy_vs_power
daily_energy.columns = ["PJM_Load_MWh"]
daily_energy.head()

## Groupby with DateTimeIndex
Using groupby with a pandas DateTimeIndex can be extremely useful and powerful.
Let's look at how this can work.

In [ ]:
indexes = [energy.index.year,
           energy.index.month,
           energy.index.week,
           energy.index.weekday,
           energy.index.day]

aggregated = energy.groupby(indexes).sum()

In [ ]:
aggregated.head()

The index level names are not particularly helpful here. We can change them.

In [56]:
# the long, ugly way
aggregated.index.set_names('year', level=0, inplace=True)
aggregated.index.set_names('month', level=1, inplace=True)
aggregated.index.set_names('week', level=2, inplace=True)
aggregated.index.set_names('weekday', level=3, inplace=True)
aggregated.index.set_names('day', level=4, inplace=True)

In [57]:
# resetting the df to get multi-index again
aggregated = energy.groupby(indexes).sum()

# the short, clean way. Both do the same thing.
index_level_names = ['year', 'month', 'week', 'weekday', 'day']
for i, index_level in enumerate(index_level_names):
            aggregated.index.set_names(index_level, level=i, inplace=True)

In [ ]:
aggregated.head(10)

## Selecting on Multi-Index: Using reset_index()
Selecting via a multi-index is difficult. It is much easier to work with these as "normal" pandas DataFrame columns.

To transform multi-indexes in to columns, we use `reset_index()`. Reset index pushes anything in the index into columns, and makes a new index with is simply an ascending count, starting at 0. 

In [ ]:
aggregated_new = aggregated.reset_index()
aggregated_new.head()

Here, we can select all rows that were in week 50 in any year.

In [ ]:
week_50 = aggregated_new.loc[aggregated_new['week']==50]
week_50.head()

## Exercise:

- Find the day of the week that averages the highest consumption

- Find the time of day that averages the lowest consumption.
    - Has this changed over the years?
    

- Is average consumption rising, falling, or staying the same over the years?
- What is the %age difference in consumption on average between April and June?

- Find the day of the week that averages the highest consumption

- Find the time of day that averages the lowest consumption.
    - Has this changed over the years?

- Is average consumption rising, falling, or staying the same over the years?

- What is the %age difference in consumption on average between April and June?